In [1]:
PATH_TO_DATA = 'input/train-balanced-sarcasm.csv'

In [2]:
# some necessary imports
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# Utility
import re
from collections import Counter
import logging
import time
import pickle
import itertools
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liyuantan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# DATASET
DATASET_ENCODING = "ISO-8859-1"

# TEXT CLENAING
TEXT_CLEANING_RE = "[^A-Za-z0-9]+"

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

train_df = pd.read_csv(PATH_TO_DATA, encoding =DATASET_ENCODING)

In [4]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [5]:
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [6]:
df = train_df[['comment', 'label']]
df.head()

,comment,label
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0


In [7]:
%%time
df.comment = df.comment.apply(lambda x: preprocess(x))

CPU times: user 23.8 s, sys: 47.2 ms, total: 23.8 s
Wall time: 23.8 s


/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
print(df.comment[1])

know west teams play west teams east teams right


In [9]:
texts = []
labels = []
texts_test = []
labels_test = []
for i in range (90000):
    texts.append(df.comment[i])
    labels.append(df.label[i])
for i in range (10000):
    texts_test.append(df.comment[i+90000])
    labels_test.append(df.label[i+90000])

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 15  # We will cut reviews after 100 words
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)  # https://keras.io/preprocessing/text/
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # This turns strings into lists of integer indices.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 46378 unique tokens.


In [11]:
d = 0
for i in range (len(sequences)):
    if len(sequences[i-d]) == 0:
        del sequences[i-d]
        del labels[i-d]
        d = d+1

In [12]:
data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

Shape of data tensor: (86775, 15)
Shape of label tensor: (86775,)


In [13]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(LSTM(8,
              dropout=0.05,
              recurrent_dropout=0.05))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 8)             80000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 80,553
Trainable params: 80,553
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(data, labels,
                    epochs=10,
                    batch_size=256,
                    validation_split = 2/9)
model.save_weights('lstm_reddit_validate.h5')

Train on 67491 samples, validate on 19284 samples
Epoch 1/10
67491/67491 [==============================] - 6s 96us/step - loss: 0.6766 - acc: 0.5749 - val_loss: 0.6612 - val_acc: 0.5974
Epoch 2/10
67491/67491 [==============================] - 5s 80us/step - loss: 0.6276 - acc: 0.6571 - val_loss: 0.6199 - val_acc: 0.6633
Epoch 3/10
67491/67491 [==============================] - 6s 82us/step - loss: 0.5943 - acc: 0.6855 - val_loss: 0.6166 - val_acc: 0.6653
Epoch 4/10
67491/67491 [==============================] - 5s 80us/step - loss: 0.5808 - acc: 0.6960 - val_loss: 0.6165 - val_acc: 0.6652
Epoch 5/10
67491/67491 [==============================] - 5s 81us/step - loss: 0.5728 - acc: 0.7023 - val_loss: 0.6191 - val_acc: 0.6654
Epoch 6/10
67491/67491 [==============================] - 6s 82us/step - loss: 0.5662 - acc: 0.7081 - val_loss: 0.6216 - val_acc: 0.6633
Epoch 7/10
67491/67491 [==============================] - 6s 89us/step - loss: 0.5618 - acc: 0.7116 - val_loss: 0.6247 - val_acc